In [2]:
## to tensorflow github page bug report

import random
import numpy as np
import pandas as pd


df_trial_train = pd.read_csv('df_trial_train_sample.txt' , sep = '\t', index_col=0) 

xyz_coor_trial_train = pd.read_csv('xyz_coor_trial_train_sample.txt',sep = '\t', index_col=0) 


n_train = len(df_trial_train)
n_valid = int(n_train*.2)


index_valid = random.sample(range(n_train)  , n_valid)
mask = np.zeros(n_train, dtype = bool)

mask[index_valid] = True



In [5]:
### Stacked AE
import matplotlib.pyplot as plt


## corr coeff:
from tensorflow.keras import backend as K
import tensorflow as tf
import tensorflow.keras as kr
import scipy as sp
def func_correlation_coefficient(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt( tf.multiply(  K.sum(K.square(xm)), K.sum(K.square(ym))  ) )
    r = r_num / r_den
#     r = sp.stats.pearsonr(x.numpy(),y.numpy())
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return r

### end corr coeff



## create root log dir for TensorBoard
import os
root_logdir = os.path.join(os.curdir, 'ruh_rootdir_logs')

def get_run_logdir(subdir_name):
    import time
    run_id = time.strftime(subdir_name+'_%Y_%m_%d_%H_%M_%S')
    return os.path.join(root_logdir, run_id)

ruh_run_logdir = get_run_logdir(subdir_name = 'stacked_ae')


my_batch_size = 128
my_lr = .001
my_ngenes_train = df_trial_train.shape[1]
my_ncube_train = df_trial_train[~mask].shape[0]
nunit_output = xyz_coor_trial_train.shape[1]

my_nepochs = 100

n_neurons = my_ngenes_train # #not iterable
n_neurons_latent = 32
n_neurons1 = n_neurons_latent * 4 #int(n_neurons/2)
n_neurons2 = n_neurons_latent * 2 #int(n_neurons/16)



## input layer
my_Input_layer = kr.layers.Input(shape=(n_neurons,)) #shape=df_trial_train.shape[1:])

## encoder:
encoder_layer = kr.layers.Dense(n_neurons1, activation = 'relu')(my_Input_layer)
encoder_layer = kr.layers.Dense(n_neurons2, activation = 'relu')(encoder_layer)

## latent space
latent_layer = kr.layers.Dense(n_neurons_latent, activation = 'relu')(encoder_layer)

## decoder:
#decoder_input = kr.layers.Input(shape=(n_neurons_latent,))
decoder_layer = kr.layers.Dense(n_neurons2 , activation = 'relu')(latent_layer)
decoder_layer = kr.layers.Dense(n_neurons1 , activation = 'relu')(decoder_layer)
decoder_layer = kr.layers.Dense(n_neurons  , activation = 'relu')(decoder_layer)

# my_outputs = kr.layers.Dense(n_neurons)(decoder_layer)

## stacked ae
stacked_ae = kr.models.Model(my_Input_layer, decoder_layer) #kr.models.Model(encoder_layer, my_outputs)
stacked_ae.summary()

## optimization + loss + metrics
myoptimizer = kr.optimizers.SGD(lr = my_lr, momentum=.9, nesterov=True) #, momentum=.01)
myoptimizer = kr.optimizers.Adam(lr = my_lr, beta_1=.9, beta_2=.999)

stacked_ae.compile(loss = 'mse', optimizer = myoptimizer,metrics=[func_correlation_coefficient])


print('#samples/epoch = ', df_trial_train[~mask].shape[0]/my_batch_size, '\n')

earlystopping_stackedAE = kr.callbacks.EarlyStopping(patience=10, restore_best_weights=True)                   
tensorboard_stackedAE = kr.callbacks.TensorBoard(ruh_run_logdir)
checkpoint_stackedAE = kr.callbacks.ModelCheckpoint('saved_models/ruh_best_stackedAE.hd5', save_best_only=True)
history = stacked_ae.fit( x = df_trial_train[~mask]
                         ,y = df_trial_train[~mask] #df_trial_train[~mask]#xyz_coor_trial_train[~mask]             
                         ,epochs=100              
                         ,validation_data=(df_trial_train[mask], df_trial_train[mask])#[df_trial_train[mask], df_trial_train[mask]]#(df_trial_train[mask], xyz_coor_trial_train[mask])              
                         ,callbacks=[ earlystopping_stackedAE
                                     ,tensorboard_stackedAE
                                     ,checkpoint_stackedAE
                                    ] 
                         ,batch_size = my_batch_size
                         , verbose = 10
                        )

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               64128     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_10 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_11 (Dense)             (None, 500)              

INFO:tensorflow:Assets written to: saved_models/ruh_best_stackedAE.hd5/assets
Epoch 4/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: saved_models/ruh_best_stackedAE.hd5/assets
Epoch 5/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attri

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: saved_models/ruh_best_stackedAE.hd5/assets
Epoch 13/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.expe

INFO:tensorflow:Assets written to: saved_models/ruh_best_stackedAE.hd5/assets
Epoch 21/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: saved_models/ruh_best_stackedAE.hd5/assets
Epoch 22/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no att

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: saved_models/ruh_best_stackedAE.hd5/assets
Epoch 30/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.expe

INFO:tensorflow:Assets written to: saved_models/ruh_best_stackedAE.hd5/assets
Epoch 39/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: saved_models/ruh_best_stackedAE.hd5/assets
Epoch 40/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no att

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: saved_models/ruh_best_stackedAE.hd5/assets
Epoch 50/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.expe

KeyboardInterrupt: 

In [1]:
class Parrot:
    
    # instance attributes
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    # instance method
    def sing(self, song):
        return "{} sings {}".format(self.name, song)

    def dance(self):
        return "{} is now dancing".format(self.name)

# instantiate the object
blu = Parrot("Blu", 10)

# call our instance methods
print(blu.sing("'Happy'"))
print(blu.dance())

Blu sings 'Happy'
Blu is now dancing


In [2]:
type(blu)
blu.age
blu.name
blu.dance()

'Blu is now dancing'

In [1]:
class Dog:
    species = "Canis familiaris"

    def __init__(self, name, age):
        self.name = name
        self.age = age

    # Instance method
    def description(self):
        return f"{self.name} is {self.age} years old"

    # Another instance method
    def speak(self, sound):
        return f"{self.name} says {sound}"


In [21]:
class Dog:
    species = "Canis familiaris"

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __str__(self):
        return f"{self.name} is {self.age} years old"

    def speak(self, sound):
        return f"{self.name} says {sound}"


In [42]:
l = [2,5,465,7]
type(l)
ll = np.array(l)
type(ll)
ll
name = 'mh'
fffirst, *mmmiddle, lllast = name

In [2]:
import tensorflow as tf
import keras

 
print(keras.__version__)

2.4.3


In [1]:
import keras
import sklearn
import tensorflow  

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

df_final = pd.read_csv('df_final.txt' , sep = '\t', index_col=0) #has NaNs


###
i = np.concatenate((np.arange(1,10), np.arange(10000,10029)))

j = np.concatenate((np.arange(1,100) , np.arange(2000,2200)))
dd = df_final.iloc[i,j]
dd.iloc[20:30,3:5] = np.nan

dd = df_final.iloc[:100,:5]
dd.iloc[:10,[0,3,2]] = np.nan


df_final = dd.copy()

###


ruh_imputer_instace = KNNImputer(missing_values=np.NaN,n_neighbors=10) #, add_indicator=True)
df_imputed = pd.DataFrame(ruh_imputer_instace.fit_transform(df_final))
df_imputed.columns = df_final.columns
df_imputed.index = df_final.index

df_final = df_imputed

df_final.to_csv('df_final_imputed_KNN10.txt', sep = '\t')


In [2]:
df_final_orig = df_final.copy()


In [3]:
i = np.concatenate((np.arange(1,10), np.arange(10000,10029)))

j = np.concatenate((np.arange(1,100) , np.arange(2000,2200)))
dd = df_final.iloc[i,j]
dd.iloc[20:30,3:5] = np.nan

dd = df_final.iloc[:100,:5]
dd.iloc[:10,[0,3,2]] = np.nan


dd

,0610010F05Rik,1110008P14Rik,1700037H04Rik,1700086L19Rik,2900026A02Rik
cube_22175,NaN,0.922821,NaN,NaN,-1.329321
cube_22176,NaN,0.909890,NaN,NaN,-1.203917
cube_22177,NaN,1.038674,NaN,NaN,-0.949789
cube_22178,NaN,0.939465,NaN,NaN,-0.628658
cube_22179,NaN,0.937714,NaN,NaN,-0.333995
...,...,...,...,...,...
cube_22415,-0.506071,0.123227,0.948489,-1.642206,-1.592179
cube_22416,-0.342819,0.207725,1.206627,-1.614210,-1.739748
cube_22417,-0.572522,0.474354,1.392126,-1.736765,-1.726678
cube_22418,-0.638065,0.511205,1.466413,-1.848312,-1.095495


In [4]:
df_final = dd.copy()


ruh_imputer_instace = KNNImputer(missing_values=np.NaN,n_neighbors=10) #, add_indicator=True)
df_imputed = pd.DataFrame(ruh_imputer_instace.fit_transform(df_final))
df_imputed.columns = df_final.columns
df_imputed.index = df_final.index

df_final = df_imputed


# df_final
df_final.to_csv('df_final_imputed_KNN10.txt', sep = '\t')


,0610010F05Rik,1110008P14Rik,1700037H04Rik,1700086L19Rik,2900026A02Rik
cube_22175,NaN,0.922821,NaN,NaN,-1.329321
cube_22176,NaN,0.909890,NaN,NaN,-1.203917
cube_22177,NaN,1.038674,NaN,NaN,-0.949789
cube_22178,NaN,0.939465,NaN,NaN,-0.628658
cube_22179,NaN,0.937714,NaN,NaN,-0.333995
...,...,...,...,...,...
cube_22415,-0.506071,0.123227,0.948489,-1.642206,-1.592179
cube_22416,-0.342819,0.207725,1.206627,-1.614210,-1.739748
cube_22417,-0.572522,0.474354,1.392126,-1.736765,-1.726678
cube_22418,-0.638065,0.511205,1.466413,-1.848312,-1.095495


,0610010F05Rik,1110008P14Rik,1700037H04Rik,1700086L19Rik,2900026A02Rik
cube_22175,-0.516685,0.922821,1.046504,-1.091627,-1.329321
cube_22176,-0.610353,0.909890,1.149992,-1.122861,-1.203917
cube_22177,-0.712613,1.038674,1.308383,-1.128450,-0.949789
cube_22178,-0.668863,0.939465,1.217440,-1.236118,-0.628658
cube_22179,-0.530819,0.937714,1.183239,-1.445041,-0.333995
...,...,...,...,...,...
cube_22415,-0.506071,0.123227,0.948489,-1.642206,-1.592179
cube_22416,-0.342819,0.207725,1.206627,-1.614210,-1.739748
cube_22417,-0.572522,0.474354,1.392126,-1.736765,-1.726678
cube_22418,-0.638065,0.511205,1.466413,-1.848312,-1.095495


In [6]:
df_final.to_csv('df_final_imputed_KNN10.txt', sep = '\t')



In [24]:
d = pd.read_csv('df_final_imputed_KNN10.txt', sep = '\t', index_col=0)
d

FileNotFoundError: [Errno 2] File df_final_imputed_KNN10.txt does not exist: 'df_final_imputed_KNN10.txt'

In [10]:
d = pd.read_csv('df_final_imputed_KNN10.txt', sep = '\t', index_col=0)
d

,0610010F05Rik,1110008P14Rik,1700037H04Rik,1700086L19Rik,2900026A02Rik
cube_22175,-0.516685,0.922821,1.046504,-1.091627,-1.329321
cube_22176,-0.610353,0.909890,1.149992,-1.122861,-1.203917
cube_22177,-0.712613,1.038674,1.308383,-1.128450,-0.949789
cube_22178,-0.668863,0.939465,1.217440,-1.236118,-0.628658
cube_22179,-0.530819,0.937714,1.183239,-1.445041,-0.333995
...,...,...,...,...,...
cube_22415,-0.506071,0.123227,0.948489,-1.642206,-1.592179
cube_22416,-0.342819,0.207725,1.206627,-1.614210,-1.739748
cube_22417,-0.572522,0.474354,1.392126,-1.736765,-1.726678
cube_22418,-0.638065,0.511205,1.466413,-1.848312,-1.095495


In [34]:
a = [np.arange(1,10), np.arange(2,20)]
b= [*a]
b
len(b)

a = np.arange(1,10)
type(a)
a = np.concatenate((np.arange(1,10), np.arange(20,29)))
type(a
    )

numpy.ndarray

In [72]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances

x = {'col1' : [3, np.nan, 5], 'col2' :  [1,0,0], 'col3':[np.nan, np.nan, 12], 'col4':[1,6, -10]}
print('typx',type(x))

x = {'col1' : [3,np.nan,np.nan,6], 'col2' :  [1,np.nan,4,5]} #,  'col3' :  [323,12,10,132]}

x = [[3,np.nan,np.nan,6], [1,np.nan,4,5]]

x = pd.DataFrame(x)

x

typx <class 'dict'>


,0,1,2,3
0,3,NaN,NaN,6
1,1,NaN,4.0,5


In [74]:
print(math.sqrt(((x.iloc[0,] - x.iloc[1,])**2).sum()))
# print(math.sqrt(((x.iloc[0,] - x.iloc[2,])**2).sum()))
# print(math.sqrt(((x.iloc[0,] - x.iloc[3,])**2).sum()))
# print(math.sqrt(((x.iloc[1,] - x.iloc[2,])**2).sum()))
# print(math.sqrt(((x.iloc[1,] - x.iloc[3,])**2).sum()))
# print(math.sqrt(((x.iloc[2,] - x.iloc[3,])**2).sum()))


2.23606797749979


In [76]:
math.sqrt(5)

2.23606797749979

In [75]:
# ruh_imputer_instace = KNNImputer(missing_values=np.NaN,n_neighbors=10) #, add_indicator=True)

# df_imputed = pd.DataFrame(ruh_imputer_instace.fit_transform(df_final))


dnan  = nan_euclidean_distances(x)
# d = euclidean_distances(x)
print('typed=' , type(d),'shape_d=', d.shape)
pd.DataFrame(dnan)

typed= <class 'numpy.ndarray'> shape_d= (4, 4)


,0,1
0,0.000000,3.162278
1,3.162278,0.000000


In [32]:
pd.DataFrame(nan_euclidean_distances(x))


,0,1,2
0,0.000000,2.0,12.961481
1,2.000000,0.0,0.000000
2,12.961481,0.0,0.000000


In [43]:
pd.DataFrame(nan_euclidean_distances(x))


,0,1,2
0,0.000000,6.244998,11.224972
1,6.244998,0.000000,19.595918
2,11.224972,19.595918,0.000000


In [21]:
fruits = ['lemon', 'pear', 'watermelon', 'tomato']

**fruits

SyntaxError: invalid syntax (<ipython-input-21-329964085a1b>, line 3)

In [ ]:
from sklearn.impute import KNNImputer




In [2]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

        
        
        
        
        


In [45]:
class MyClass:
    def method(self):
        return 'instance method called', self

    @classmethod
    def classmethod(cls):
        return 'class method called', cls

    @staticmethod
    def staticmethod():
        return 'static method called'


In [54]:
a = MyClass

a.method(a)

a.classmethod()
# a.staticmethod()

('class method called', __main__.MyClass)

In [13]:
a = Dog('sdf',9)
type(a)

__main__.Dog

In [20]:
a.age
a.speak('asdasfd')
a.description()


'sdf is 9 years old'

In [1]:
import numpy as np
import pandas as pd

In [2]:
import SpXPSmall
dd = SpXPSmall.func_df_spatial_xp(p = .8, int_bool=True)
dd

rank-based inverse normal-transformation CALLED!
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toBe3
DONE3
DONE4
toBe1
DONE1
toBe2
DONE2
toB

,cube_22175,cube_22176,cube_22177,cube_22178,cube_22179,cube_22180,cube_22181,cube_22182,cube_22183,cube_22184,...,cube_22252,cube_22253,cube_22290,cube_22291,cube_22292,cube_22293,cube_22294,cube_22295,cube_22296,cube_22297
x,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
y,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,...,14.000000,14.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
z,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000,26.000000,27.000000,28.000000,...,38.000000,39.000000,18.000000,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000
Inpp5a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Snap47,2.156356,2.166107,2.166107,1.757763,1.524945,1.215324,1.353934,1.353934,1.353934,1.215324,...,0.650837,0.650837,2.156356,2.166107,2.166107,1.215324,0.897041,0.809580,0.809580,0.897041
Nvl,-0.631138,-0.577449,-0.507048,-0.577449,-0.577449,-0.577449,-0.650837,-0.897041,-0.809580,-0.439077,...,-0.183322,-0.439077,-0.975742,-0.727913,-0.308666,-0.121834,-0.183322,-0.183322,-0.121834,-0.121834
Gtf3c2,-0.156326,0.000000,0.000000,-0.060804,-0.121834,-0.060804,-0.245511,-0.373077,-0.245511,-0.060804,...,-1.096804,-1.353934,0.156326,0.183322,0.060804,-0.183322,-0.439077,-0.507048,-1.096804,-0.809580
Acadl,-1.512390,-1.524945,-1.524945,-1.757763,-1.215324,-1.215324,-1.215324,-1.096804,-0.650837,-1.096804,...,-0.809580,-0.809580,-1.081436,-1.215324,-1.353934,-0.991970,-0.507048,-0.439077,-0.373077,-0.577449
Baiap3,0.031143,-0.507048,-0.727913,-0.650837,-0.650837,-0.809580,-0.809580,-0.650837,-0.439077,-0.373077,...,-2.166107,-2.166107,-0.284017,-0.121834,0.000000,0.507048,0.308666,0.121834,0.121834,0.060804
S100a6,-0.879958,-1.096804,-1.096804,-1.215324,-1.757763,-2.166107,-1.353934,-1.215324,-0.991970,-1.215324,...,-1.353934,-1.096804,-0.791639,-0.991970,-1.215324,-1.757763,-1.757763,-1.757763,-1.757763,-1.757763


In [16]:
dd

,cube_22175,cube_22176,cube_22177,cube_22178,cube_22179,cube_22180,cube_22181,cube_22182,cube_22183,cube_22184,...,cube_22252,cube_22253,cube_22290,cube_22291,cube_22292,cube_22293,cube_22294,cube_22295,cube_22296,cube_22297
x,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
y,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,...,14.000000,14.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
z,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000,26.000000,27.000000,28.000000,...,38.000000,39.000000,18.000000,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000
Inpp5a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Snap47,2.156356,2.166107,2.166107,1.757763,1.524945,1.215324,1.353934,1.353934,1.353934,1.215324,...,0.650837,0.650837,2.156356,2.166107,2.166107,1.215324,0.897041,0.809580,0.809580,0.897041
Nvl,-0.631138,-0.577449,-0.507048,-0.577449,-0.577449,-0.577449,-0.650837,-0.897041,-0.809580,-0.439077,...,-0.183322,-0.439077,-0.975742,-0.727913,-0.308666,-0.121834,-0.183322,-0.183322,-0.121834,-0.121834
Gtf3c2,-0.156326,0.000000,0.000000,-0.060804,-0.121834,-0.060804,-0.245511,-0.373077,-0.245511,-0.060804,...,-1.096804,-1.353934,0.156326,0.183322,0.060804,-0.183322,-0.439077,-0.507048,-1.096804,-0.809580
Acadl,-1.512390,-1.524945,-1.524945,-1.757763,-1.215324,-1.215324,-1.215324,-1.096804,-0.650837,-1.096804,...,-0.809580,-0.809580,-1.081436,-1.215324,-1.353934,-0.991970,-0.507048,-0.439077,-0.373077,-0.577449
Baiap3,0.031143,-0.507048,-0.727913,-0.650837,-0.650837,-0.809580,-0.809580,-0.650837,-0.439077,-0.373077,...,-2.166107,-2.166107,-0.284017,-0.121834,0.000000,0.507048,0.308666,0.121834,0.121834,0.060804
S100a6,-0.879958,-1.096804,-1.096804,-1.215324,-1.757763,-2.166107,-1.353934,-1.215324,-0.991970,-1.215324,...,-1.353934,-1.096804,-0.791639,-0.991970,-1.215324,-1.757763,-1.757763,-1.757763,-1.757763,-1.757763


In [4]:
dd.to_csv('dd.txt', sep ='\t')


In [15]:
gg = pd.read_csv('dd.txt' , sep = '\t', index_col=0)
gg

,cube_22175,cube_22176,cube_22177,cube_22178,cube_22179,cube_22180,cube_22181,cube_22182,cube_22183,cube_22184,...,cube_22252,cube_22253,cube_22290,cube_22291,cube_22292,cube_22293,cube_22294,cube_22295,cube_22296,cube_22297
x,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
y,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,...,14.000000,14.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
z,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000,26.000000,27.000000,28.000000,...,38.000000,39.000000,18.000000,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000
Inpp5a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Snap47,2.156356,2.166107,2.166107,1.757763,1.524945,1.215324,1.353934,1.353934,1.353934,1.215324,...,0.650837,0.650837,2.156356,2.166107,2.166107,1.215324,0.897041,0.809580,0.809580,0.897041
Nvl,-0.631138,-0.577449,-0.507048,-0.577449,-0.577449,-0.577449,-0.650837,-0.897041,-0.809580,-0.439077,...,-0.183322,-0.439077,-0.975742,-0.727913,-0.308666,-0.121834,-0.183322,-0.183322,-0.121834,-0.121834
Gtf3c2,-0.156326,0.000000,0.000000,-0.060804,-0.121834,-0.060804,-0.245511,-0.373077,-0.245511,-0.060804,...,-1.096804,-1.353934,0.156326,0.183322,0.060804,-0.183322,-0.439077,-0.507048,-1.096804,-0.809580
Acadl,-1.512390,-1.524945,-1.524945,-1.757763,-1.215324,-1.215324,-1.215324,-1.096804,-0.650837,-1.096804,...,-0.809580,-0.809580,-1.081436,-1.215324,-1.353934,-0.991970,-0.507048,-0.439077,-0.373077,-0.577449
Baiap3,0.031143,-0.507048,-0.727913,-0.650837,-0.650837,-0.809580,-0.809580,-0.650837,-0.439077,-0.373077,...,-2.166107,-2.166107,-0.284017,-0.121834,0.000000,0.507048,0.308666,0.121834,0.121834,0.060804
S100a6,-0.879958,-1.096804,-1.096804,-1.215324,-1.757763,-2.166107,-1.353934,-1.215324,-0.991970,-1.215324,...,-1.353934,-1.096804,-0.791639,-0.991970,-1.215324,-1.757763,-1.757763,-1.757763,-1.757763,-1.757763


In [14]:
pd.DataFrame.dropna(dd) == pd.DataFrame.dropna(gg)

,cube_22175,cube_22176,cube_22177,cube_22178,cube_22179,cube_22180,cube_22181,cube_22182,cube_22183,cube_22184,...,cube_22252,cube_22253,cube_22290,cube_22291,cube_22292,cube_22293,cube_22294,cube_22295,cube_22296,cube_22297
x,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
y,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
z,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
Snap47,False,False,False,True,True,True,True,True,True,True,...,True,True,False,False,False,True,True,True,True,True
Nvl,True,True,True,True,True,True,True,True,True,False,...,True,False,False,True,False,False,True,True,False,False
Gtf3c2,True,True,True,False,False,False,False,False,False,False,...,False,True,True,True,False,True,False,True,False,True
Acadl,False,True,True,False,True,True,True,False,True,False,...,True,True,False,True,True,True,True,False,False,True
Baiap3,True,True,True,True,True,True,True,True,False,False,...,True,True,True,False,True,True,False,False,False,False
S100a6,True,False,False,True,False,True,True,True,True,True,...,True,False,True,True,True,False,False,False,False,False
Clock,True,True,True,True,True,True,True,True,True,True,...,False,True,True,True,False,True,True,True,False,False


In [5]:
dnorm = SpXPSmall.func_df_spatial_xp(p = .8, int_bool=False)

Normalization CALLED!
Normalization DONE!


In [6]:
dnorm

,cube_22175,cube_22176,cube_22177,cube_22178,cube_22179,cube_22180,cube_22181,cube_22182,cube_22183,cube_22184,...,cube_156399,cube_156400,cube_156401,cube_156402,cube_156403,cube_156404,cube_156405,cube_156406,cube_156407,cube_156408
x,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000
y,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,...,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
z,19.000000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000,26.000000,27.000000,28.000000,...,31.000000,32.000000,33.000000,34.000000,35.000000,36.000000,37.000000,38.000000,39.000000,40.000000
Inpp5a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.223306,-0.351236,-0.209369,0.214584,0.357507,0.348156,0.066595,-0.401773,-0.593920,-0.628171
Snap47,-0.705635,-0.068064,0.161530,0.309174,0.594719,0.596155,0.396617,0.136030,-0.267496,-0.724219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nvl,-0.763969,-0.733267,-0.672886,-0.647366,-0.624522,-0.597670,-0.604393,-0.650270,-0.703134,-0.678266,...,-0.548416,-0.538679,-0.231981,-0.179781,-0.532102,-0.656248,-0.701611,-0.737862,-0.762724,-0.775947
Gtf3c2,-0.463107,-0.260661,-0.034864,0.103620,0.103521,0.047086,-0.022744,-0.106425,-0.153277,-0.184886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acadl,-0.468431,-0.467550,-0.463314,-0.454229,-0.442743,-0.438721,-0.440780,-0.421694,-0.389259,-0.428114,...,0.213506,0.337790,0.805833,1.724506,1.824773,0.624629,-0.123029,-0.303839,-0.344606,-0.393602
Baiap3,-0.389214,-0.396264,-0.384539,-0.358811,-0.355388,-0.349984,-0.352341,-0.331617,-0.315261,-0.342315,...,-0.262249,-0.276089,-0.207817,-0.031285,0.345030,0.872815,0.918046,0.674546,0.194351,-0.237687
S100a6,-0.187643,-0.169323,-0.140690,-0.084279,-0.167760,-0.189393,-0.158311,-0.092967,-0.047650,-0.104144,...,-0.168427,-0.166703,-0.120781,-0.116065,-0.122513,-0.037847,0.068924,0.169152,0.040729,-0.153627
